In [1]:
#import run_rsa_demo, run_elgamal_demo, run_ecc_demo_with_strings
from RSA import run_rsa_demo
from Elgamal import run_elgamal_demo
from ECC import run_ecc_demo_with_strings
import tqdm

In [2]:
# read input.txt and store each line in a variable

sentences = []
with open("input.txt", "r") as file:
    sentences = file.readlines()
    

In [3]:
rsa_key_gen_times = []
elgamal_key_gen_times = []
ecc_key_gen_times = []
rsa_encrypt_times = []
elgamal_encrypt_times = []
ecc_encrypt_times = []
rsa_decrypt_times = []
elgamal_decrypt_times = []
ecc_decrypt_times = []

rsa_total_times = []
elgamal_total_times = []
ecc_total_times = []


In [4]:
def store_in_arr(algo, key_gen_time, encrypt_time, decrypt_time, total_time):
    if algo == "RSA":
        rsa_key_gen_times.append(key_gen_time)
        rsa_encrypt_times.append(encrypt_time)
        rsa_decrypt_times.append(decrypt_time)
        rsa_total_times.append(total_time)
    elif algo == "Elgamal":
        elgamal_key_gen_times.append(key_gen_time)
        elgamal_encrypt_times.append(encrypt_time)
        elgamal_decrypt_times.append(decrypt_time)
        elgamal_total_times.append(total_time)
    elif algo == "ECC":
        ecc_key_gen_times.append(key_gen_time)
        ecc_encrypt_times.append(encrypt_time)
        ecc_decrypt_times.append(decrypt_time)
        ecc_total_times.append(total_time)

In [5]:
# pass each sentence to the demo functions
from concurrent.futures import ThreadPoolExecutor, as_completed
import tqdm

def process_sentence(sentence):
    sentence = sentence.strip()

    # RSA
    key_gen_time_rsa, encrypt_time_rsa, decrypt_time_rsa, total_time_rsa = run_rsa_demo(sentence)
    store_in_arr("RSA", key_gen_time_rsa, encrypt_time_rsa, decrypt_time_rsa, total_time_rsa)

    # ElGamal
    key_gen_time_elgamal, encrypt_time_elgamal, decrypt_time_elgamal, total_time_elgamal = run_elgamal_demo(sentence)
    store_in_arr("Elgamal", key_gen_time_elgamal, encrypt_time_elgamal, decrypt_time_elgamal, total_time_elgamal)

    # ECC
    key_gen_time_ecc, encrypt_time_ecc, decrypt_time_ecc, total_time_ecc = run_ecc_demo_with_strings(sentence)
    store_in_arr("ECC", key_gen_time_ecc, encrypt_time_ecc, decrypt_time_ecc, total_time_ecc)

    return True


def run_on_inputs_parallel(sentences, max_workers=8):
    futures = []
    counter = 0
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for sentence in sentences:
            futures.append(executor.submit(process_sentence, sentence))

        # tqdm progress bar over completed tasks
        for _ in tqdm.tqdm(as_completed(futures), total=len(futures), desc="Processing sentences"):
            pass


In [6]:
# in a csv file, 1st column is the sentence, 2nd is rsa key gen time, 3rd is rsa encrypt time, 4th is rsa decrypt time, 5th is rsa total time
# 6th is elgamal key gen time, 7th is elgamal encrypt time, 8th is elgamal decrypt time, 9th is elgamal total time
# 10th is ecc key gen time, 11th is ecc encrypt time, 12th is ecc decrypt time, 13th is ecc total time

import csv
def write_to_csv():
    with open("benchmark_results.csv", "w", newline="") as csvfile:
        csvwriter = csv.writer(csvfile)
        # write header
        csvwriter.writerow(["Sentence", 
                            "RSA Key Gen Time", "RSA Encrypt Time", "RSA Decrypt Time", "RSA Total Time",
                            "Elgamal Key Gen Time", "Elgamal Encrypt Time", "Elgamal Decrypt Time", "Elgamal Total Time",
                            "ECC Key Gen Time", "ECC Encrypt Time", "ECC Decrypt Time", "ECC Total Time"])
        # write data
        for i in range(len(sentences)):
            csvwriter.writerow([sentences[i].strip(),
                                rsa_key_gen_times[i], rsa_encrypt_times[i], rsa_decrypt_times[i], rsa_total_times[i],
                                elgamal_key_gen_times[i], elgamal_encrypt_times[i], elgamal_decrypt_times[i], elgamal_total_times[i],
                                ecc_key_gen_times[i], ecc_encrypt_times[i], ecc_decrypt_times[i], ecc_total_times[i]])



In [7]:
run_on_inputs_parallel(sentences, max_workers=8)
write_to_csv()

Processing sentences: 100%|██████████| 7500/7500 [4:51:31<00:00,  2.33s/it]   
